In [1]:
# Install the required extra libraries.
#!pip install pandas-profiling
#!pip install pandas-gbq

In [24]:
# Import useful libraries.
import pandas as pd
import numpy as np
import pandas_profiling
import pandas_gbq
import logging

## Explore the data

In [3]:
# Load data
populations_data = pd.read_csv('https://pkgstore.datahub.io/core/population-city/unsd-citypopulation-year-fm_csv/data/8b8d4d72dbf1a503bc3e241b394ce301/unsd-citypopulation-year-fm_csv.csv')

In [4]:
populations_data.head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2013,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5445,NaN
1,Åland Islands,2013,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5925,NaN
2,Åland Islands,2012,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5408,NaN
3,Åland Islands,2012,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5896.5,NaN
4,Åland Islands,2011,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2012,5363.5,NaN


In [5]:
# Conduct all the basic necessary exploratory analysis with pandas_profiling library.
profile = populations_data.profile_report()
profile

In [25]:
profile.to_file(output_file="populations_data_exploratory_profile.html")

## Clean the Data

In [7]:
# There are 163 missing data points in the Value column so let's exclude them and verify.
print(f"There are {populations_data.shape[0]} rows.")
populations_data = populations_data[pd.isna(populations_data['Value'])==False]
print(f"After excluding missing values, there are {populations_data.shape[0]} rows.")

There are 28528 rows.
After excluding missing values, there are 28365 rows.


In [8]:
# Include only rows with numeric 'Year' and 'Value' values.
populations_data = populations_data[populations_data.Year.apply(lambda x: x.isnumeric())]
populations_data = populations_data[populations_data.Value.apply(lambda x: x.isnumeric())]
print(f"After excluding rows without population values and incorrect Year values, there are {populations_data.shape[0]} rows.")

After excluding rows without population values and incorrect Year values, there are 27458 rows.


In [10]:
# Lowercase all categorical features.
columns_to_lowercase = ['Country_or_Area', 'Area', 'Sex', 'City', 'City_type', 'Record_Type', 'Reliability']

populations_data[columns_to_lowercase] = populations_data[columns_to_lowercase].apply(lambda x: x.str.lower())


In [11]:
# Set numeric columns to numeric type.
populations_data['Year'] = pd.to_numeric(populations_data['Year'])
populations_data['Value'] = pd.to_numeric(populations_data['Value'])


In [12]:
populations_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27458 entries, 0 to 28363
Data columns (total 11 columns):
Country_or_Area    27458 non-null object
Year               27458 non-null int64
Area               27458 non-null object
Sex                27458 non-null object
City               27458 non-null object
City_type          27458 non-null object
Record_Type        27458 non-null object
Reliability        27458 non-null object
Source_Year        27458 non-null object
Value              27458 non-null int64
Value_Footnotes    9537 non-null object
dtypes: int64(2), object(9)
memory usage: 2.5+ MB


In [13]:
populations_data.Area.value_counts()

total    27458
Name: Area, dtype: int64

In [14]:
# Since 'Area' has only one value for all observations, we'll exclude that from the dataset.
populations_data.drop('Area', inplace = True, axis = 1)
# We're note going to use Value Footnotes in our analysis so we'll exclude that.
populations_data.drop('Value_Footnotes', inplace = True, axis = 1)

 Since some cities for some years have estimates from multiple years (Source Year) then choose the latest figure for each city and Year combination.\
 This means that for some city-year combinations we are considering only an estimate and not an official 


In [15]:
# First sort the dataframe accordingly.
populations_data.sort_values(by = 'Source_Year', ascending = False, inplace=True)

# Remove duplicates by city-year combinations
populations_data.drop_duplicates(subset=['Country_or_Area', 'City', 'Year', 'Sex'], keep = 'first', inplace=True)

In [16]:
# Remove spaces from variable names to be able to upload the dataset to BigQuery.
#populations_data.columns = populations_data.columns.str.replace(' ', '')
populations_data.columns

Index(['Country_or_Area', 'Year', 'Sex', 'City', 'City_type', 'Record_Type',
       'Reliability', 'Source_Year', 'Value'],
      dtype='object')

In [23]:
# Align the casing of City_type column.
# Rename 'Value' to 'Population' to make the variable meaning more obvious.
populations_data.rename(columns={'City_type':'City_Type',
                                'Value':'Population'}, inplace=True)

## To understand how evenly a given country's population is spread, calculate the Gini Index.

In [18]:
# First group populations by cities countries and years.
populations_data_by_cities = populations_data.groupby(['Country_or_Area','City','Year']).sum()
populations_data_by_cities

Population
Country_or_Area City             Year            
albania         durrës           2011      113249
                tirana           2003      392863
                                 2011      418495
american samoa  pago pago        2000        4278
andorra         andorra la vella 2003       21245
...                                           ...
åland islands   mariehamn        2009       11064
                                 2010        5327
                                 2011        5863
                                 2012        5408
                                 2013       11370

[11312 rows x 1 columns]

In [19]:
def gini(x):
    """
    Calculate the Gini index for a numeric array
    :param x:  one-dimensional numeric array
    :return g: float, the gini index
    """
    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [20]:
# Declare variables.
country_vector = []
year_vector = []
country_population_vector = []
gini_vector = []

# Loop through all the Country-Year combinations and calculate their Gini coefficients.
for index, row in populations_data[['Country_or_Area', 'Year']].drop_duplicates().iterrows():
    
    country_data = populations_data_by_cities.xs(row[0], level='Country_or_Area')
    country_year_data = country_data.xs(row[1], level='Year')
    gini_country_year = gini(country_year_data['Population'])
    
    country_vector.append(row[0])
    year_vector.append(row[1])
    country_population_vector.append(country_year_data['Population'].sum())
    gini_vector.append(gini_country_year)
    

/Users/Edgar/.pyenv/versions/3.7.4/envs/ELSA/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: outer method for ufunc <ufunc 'subtract'> is not implemented on pandas objects. Returning an ndarray, but in the future this will raise a 'NotImplementedError'. Consider explicitly converting the Series to an array with '.array' first.
  
/Users/Edgar/.pyenv/versions/3.7.4/envs/ELSA/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [21]:
# Enter new data into a dataframe.1
gini_dataframe_by_country_year = pd.DataFrame({'country':country_vector,
                                              'year':year_vector,
                                               'country_population':country_population_vector,
                                              'city_pop_gini_coeficient':gini_vector})
gini_dataframe_by_country_year.tail()

,country,year,country_population,city_pop_gini_coeficient
612,united republic of tanzania,1988,2638821,0.465622
613,pitcairn,1987,60,0.000000
614,pitcairn,1986,55,0.000000
615,pitcairn,1985,66,0.000000
616,lebanon,1970,474870,0.000000


## Upload the datasets to Bigquery for Data Studio reporting

In [22]:
# Set uploader logger to show debugging messages.
gbq_logger = logging.getLogger('pandas_gbq')
gbq_logger.setLevel(logging.DEBUG)

# I'm using my personal project 'brave-tech' for hosting the dataset.
populations_data.to_gbq('ELSA_dataset.census_data',
    project_id='brave-tech',
    if_exists='replace')

gini_dataframe_by_country_year.to_gbq('ELSA_dataset.census_data_gini_coeffs_by_country',
    project_id='brave-tech',
    if_exists='replace')

Google Data Studio report is available here: https://datastudio.google.com/u/0/reporting/1qXpSwr99bAYhKVUu6LBQiT8MQlQ3egv1/page/KlT7